In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import transforms
import numpy as np
import matplotlib.pyplot as plt

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
BATCH_SIZE= 64
EPOCHS = 50
LATENT_DIM = 100

In [ ]:
transform  = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))]
)
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
#생성자
#100차원 노이즈를 입력받아서  28x28x1 출력력
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(LATENT_DIM, 256 * 7 * 7),
            nn.ReLU(),
            nn.Unflatten(1, (256, 7, 7)),

            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            nn.ConvTranspose2d(64, 1, kernel_size=3, stride=1, padding=1),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        return img  

generator = Generator().to(DEVICE)

In [ ]:
#판별자 
#28x28x1 크기를 입력 받아서 해당 이미지가 진짜일 확률 
class Disriminator(nn.Module):
    def __init__(self):
        super(Disriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Flatten(),
            nn.Linear(128 * 7 * 7, 1)
        )

    def forward(self, img):
        validity = self.model(img)
        return validity
   
discriminator = Disriminator().to(DEVICE)

In [ ]:
#손실함수 및 옵티마이저 정의 
# 손실함수
adversarial_loss = nn.BCEWithLogitsLoss()

# 옵티마이저
optimizer_G = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

In [ ]:
def view_images(epoch, generator, latent_dim, device):
    with torch.no_grad():
        z = torch.randn(16, latent_dim).to(device)
        generated = generator(z).cpu()

        fig, axes = plt.subplots(4,4, figsize=(4,4))
        for i, ax in enumerate(axes.flat):
            img = generated[i] * 0.5 + 0.5
            ax.imshow(img.squeeze(), cmap='gray')
            ax.axis('off')

        plt.savefig(f"image_at_epoch_{epoch:04d}.png")
        plt.show()

In [ ]:
for epoch in range(EPOCHS):
    for i, (imgs, _) in enumerate(train_loader):
       
        # 실제 이미지와 가짜 이미지에 대한 레이블 생성
        real_labels = torch.ones(imgs.size(0), 1).to(DEVICE)
        fake_labels = torch.zeros(imgs.size(0), 1).to(DEVICE)
       
        # 실제 이미지를 장치(GPU/CPU)로 이동
        real_imgs = imgs.to(DEVICE)
       
        # ---------------------
        #  생성자(Generator) 학습
        # ---------------------
        optimizer_G.zero_grad()
       
        # 노이즈를 샘플링하여 가짜 이미지 생성
        z = torch.randn(imgs.size(0), LATENT_DIM).to(DEVICE)
        generated_imgs = generator(z)
       
        # 생성자 손실 계산 (판별자를 속이도록)
        g_loss = adversarial_loss(discriminator(generated_imgs), real_labels)
       
        # 생성자 역전파 및 가중치 업데이트
        g_loss.backward()
        optimizer_G.step()
       
        # ---------------------
        #  판별자(Discriminator) 학습
        # ---------------------
        optimizer_D.zero_grad()
       
        # 실제 이미지에 대한 손실 계산
        real_loss = adversarial_loss(discriminator(real_imgs), real_labels)
       
        # 가짜 이미지에 대한 손실 계산
        fake_loss = adversarial_loss(discriminator(generated_imgs.detach()), fake_labels)
        # .detach()를 사용하여 생성자의 그래디언트가 판별자 학습에 영향을 주지 않도록 함
       
        d_loss = (real_loss + fake_loss) / 2
       
        # 판별자 역전파 및 가중치 업데이트
        d_loss.backward()
        optimizer_D.step()

    # 에포크 종료 후 로그 출력
    print(f"[Epoch {epoch}/{EPOCHS}] [D loss: {d_loss.item():.4f}] [G loss: {g_loss.item():.4f}]")
    view_images(epoch, generator, LATENT_DIM, DEVICE)